### En este notebook comparararemos los antecedentes del comjunto de test que han sido anotados por humanos con los que identifique el modelo que hemos creado

In [1]:
import json
from glob import glob
import spacy
import pandas as pd
from spacy.lang.es import Spanish
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span
from spacy import displacy
from medspacy.target_matcher import TargetMatcher, TargetRule

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=ResourceWarning)

Cargamos el modelo entrenado

In [2]:
nlp = spacy.load("./models/model_antecedentes_ner_es_md")

Añadimos al modelo el componente que añade los códigos CEI10 a los antecedentes que encuentre

In [3]:
target_matcher = TargetMatcher(nlp)
Span.set_extension("CIE10", default="", force=True)
rules = TargetRule.from_json("./data/cie10_rules.jsonl")
target_matcher.add(rules)
nlp.add_pipe(target_matcher)
nlp.pipe_names

['tagger', 'parser', 'entity_ruler', 'ner', 'target_matcher']

Aplicamos el modelo a los archivos del conjunto de test

In [4]:
# directorio dónde están cada uno de los txt que contienen un informe para testear el modelo
read_files = glob("./data/test/text_files/*.txt") 

# generamos un archivo json que contine el nombre de cada archivo con los antedentes y sus códigos encontrados por el modelo
with open("data/antecedentes_test.jsonl", "w", encoding="utf-8" ) as outfile: 
    archivo_antec=[]
    for f in read_files:
        with open(f, "r", encoding="utf-8") as infile:
            nombre_archivo = f
            nombre_archivo = nombre_archivo.replace("./data/test/text_files\\", "")
            nombre_archivo = nombre_archivo.replace(".txt", "")
            text = infile.read()
            nuevo_informe = nlp(text)
            ante_encontrados = nombre_archivo, [[ent._.CIE10, ent.text] for ent in nuevo_informe.ents]
            archivo_antec.append(ante_encontrados)
            json.dump(ante_encontrados, outfile, indent=1)

Generamos una lista que contenga el nombre de archivo, código y antecedente por cada antecedente encontrado. Con esto lo hacemos para coincidir con el formato de los antecedentes anotados

In [5]:
antec_encontrados = [[archivo, *antec] for archivo, lista in archivo_antec for antec in lista]

Creamos un dataset a partir de los resultados de antecedentes y códigos que ha encontrado nuestro modelo 

In [6]:
df = pd.DataFrame(antec_encontrados)
df.columns = ["Archivo", "CEI10", "Antecedente"]
df

,Archivo,CEI10,Antecedente
0,S0004-06142005000500011-1,E11.9,Diabetes Mellitus tipo II
1,S0004-06142005000500011-1,F10.20,enolismo
2,S0004-06142005000500011-1,F17.210,fumador
3,S0004-06142005000500011-1,C07,carcinoma
4,S0004-06142005000900014-1,,leiomioma vesical
...,...,...,...
565,S2254-28842014000300010-1,E78.5,dislipemia
566,S2254-28842014000300010-1,I25.9,cardiopatía isquémica
567,S2254-28842014000300010-1,I21.3,infarto agudo de miocardio
568,S2254-28842014000300010-1,N18.9,Insuficiencia renal crónica


In [7]:
# lo guardamos en un archivo excel
df.to_excel('data/test/Codigos_Test_Encontrados_Con_Antecedente.xlsx')

In [8]:
# eliminamos las filas que no tienen CEI10
df2 = (pd.read_excel("./data/test/Codigos_Test_Encontrados_Con_Antecedente.xlsx", usecols=['Archivo', 'CEI10', 'Antecedente']))
df2 = df2.dropna()
df2.to_excel('data/test/Codigos_Test_Encontrados_Con_Antecedente_Final.xlsx')
df2

,Archivo,CEI10,Antecedente
0,S0004-06142005000500011-1,E11.9,Diabetes Mellitus tipo II
1,S0004-06142005000500011-1,F10.20,enolismo
2,S0004-06142005000500011-1,F17.210,fumador
3,S0004-06142005000500011-1,C07,carcinoma
5,S0004-06142006000100010-1,I10,hipertensa
...,...,...,...
565,S2254-28842014000300010-1,E78.5,dislipemia
566,S2254-28842014000300010-1,I25.9,cardiopatía isquémica
567,S2254-28842014000300010-1,I21.3,infarto agudo de miocardio
568,S2254-28842014000300010-1,N18.9,Insuficiencia renal crónica


Creamos un dataset a partir del archivo de test con los antecedentes y códigos anotados 

In [10]:
# hacemos las modificaciones necesarioas para que coincida con el formato de los encontrados
df3 = (pd.read_excel("./data/test/Codigos_Test_Anotados_Con_Posicion_Original.xlsx", usecols=['Archivo', 'CEI10', 'Antecedente']))
df3['CEI10'] = df3['CEI10'].str.upper()
df3.to_excel('data/test/Codigos_Test_Anotados_Con_Antecedente_Final.xlsx')
df3

,Archivo,CEI10,Antecedente
0,S0004-06142005000500011-1,C67.9,carcinoma urotelial superficial de vejiga
1,S0004-06142005000500011-1,E11.9,Diabetes Mellitus tipo II
2,S0004-06142005000500011-1,F10.20,enolismo activo
3,S0004-06142005000500011-1,F17.210,cigarrillos
4,S0004-06142006000100010-1,E11.9,diabetes
...,...,...,...
605,S2254-28842014000300010-1,I21.3,infarto agudo de miocardio
606,S2254-28842014000300010-1,I25.2,antecedentes de infarto agudo de miocardio
607,S2254-28842014000300010-1,I25.9,cardiopatía isquémica
608,S2254-28842014000300010-1,N18.9,Insuficiencia renal crónica secundaria a nefro...


En el archivo Evaluacion_Anotados_Encontrados.xlsx hemos hecho una comparación entre los antecedentes anotados los encontrados por el modelo